In [22]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
import os 
import matplotlib.pyplot as plt
import matplotlib as mpl 
import time
import functools 
import IPython.display as display 
import PIL.Image 


In [ ]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



**TASK: **
>Transforming an Image to look like a Monet Painting 
>Generative Adversarial Networks -

** Code Workflow and Inital Notes **
>Want to create this in tensorFlow -> Good Practice Problem 

V0.1 
>Create a program that is able to to do basic neural style transfer following tensor flow documentation
>This is not optimized and there is a better way of doing this similar to cyclegan which generates an image in a particular style rather than transfering the style over 

**GAN Notes **
>Generative Adversarial Networks -> Two Models trained simultaneously 
>Generator (Artist Network) -> Learns to create images that look real
>Discriminator (Critic Network) -> Learns to tell real images apart from fakes 
> More Clearly We are trying to neural style transfer referenced in Gatys et al. 

* Coding Notes & Structure Notes * 
>Rather than getting into direct coding define tasks that will be repeated multiple times throughout the process and convert them into functions 

Definition: Tensor to Image 
>Takes in a tensor and returns an image 

Notes:
PIL.Image -> Python Imaging Library > Image class a
>General Image Editing Capabilities, in this case we are using PIL.Image.fromarray - > to turn an array(our tensor turned into np.array) into an image 

>tensor = tensor*255 -> Our tensor will most likely be normalized due to easier processsing so we will need to unnormalize 

>dont know what the assert is doing exactly but it seems to be reshaping tensor to an array format for .fromarray code 

In [24]:
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype = np.uint8)
    if np.ndim(tensor>3):
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

Definition : Load an Image from Path and limit its dimension to 512 pixels 

In [25]:
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels = 3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim/long_dim
    
    new_shape = tf.cast(shape*scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

Definition: Imshow -> Pretty obvious haha

In [26]:
def imshow(image, title = None):
    if len(image.shape)>3:
        image = tf.squeeze(image, axis = 0)
    
    plt.imshow(image)
    if title:
        plt.title(title)

Code Snippet to Load Two Images 
>We will choose one from content and one from style

In [27]:
style_image = load_img("../input/gan-getting-started/monet_jpg/000c1e3bff.jpg")
content_img = load_img("../input/gan-getting-started/photo_jpg/00068bc07f.jpg")

plt.subplot(1,2,1)
imshow(style_image, "Monet Style")

plt.subplot(1,2,2)
imshow(content_img, "Photo to Be Converted")

In [28]:
vgg = tf.keras.applications.VGG19(include_top = False, weights = 'imagenet')

print()
for layer in vgg.layers:
    print(layer.name)

The above code represents a already created network from tensor flow, CNN for image classification, we removed the classification layer and will now choose intermediate layers to represent style and content of the image 

In [29]:
content_layers = ['block5_conv2']

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']

Definition: Creating a Pretrained Model 

In [33]:
def vgg_layers(layer_names):
    vgg = tf.keras.applications.VGG19(include_top = False, weights = 'imagenet')
    vgg.trainable = False
    
    outputs = [vgg.get_layer(name).output for name in layer_names]
    model = tf.keras.Model([vgg.input], outputs)
    return model

In [35]:
style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(style_image*255)

for name, output in zip(style_layers, style_outputs):
    print(name)
    print(" Shape: ", output.numpy().shape)
    print(" min: ", output.numpy().min)
    print("max: ", output.numpy().max)
    print("mean: ", ouput.numpy().mean)
    print()
    
